# DEPENDÊNCIAS

In [1]:
import os
import pandas as pd
import numpy as np

from google.colab import drive

# FUNÇÕES

## Exibição

In [2]:
def df_show_head(df: pd.DataFrame, n: int = 5) -> None:
    display(df.head(n))
    print(f"Shape: {df.shape}")

## Salvamento

In [3]:
def concatenate_csvs(file_paths: list[str], output_path: str, verbose: bool = True) -> None:
    """
    Lê e concatena arquivos CSV, salvando o resultado em um único arquivo.

    :param file_paths: Lista com os caminhos dos arquivos CSV de entrada.
    :param output_path: Caminho para salvar o arquivo CSV concatenado.
    """
    dataframes = []

    for path in file_paths:
        try:
            df = pd.read_csv(path)
            dataframes.append(df)

            if verbose:
                print(f"CSV carregado: {path}")

        except Exception as e:
            print(f"Erro ao ler {path}: {e}")

    if dataframes:
        df_final = pd.concat(dataframes, ignore_index=True)
        df_final.to_csv(output_path, index=False)
        print(f"Arquivo final salvo em: {output_path}")

    else:
        print("Nenhum arquivo válido foi carregado.")

def concatenate_excels(file_paths: list[str], output_path: str) -> None:
    """
    Reads and concatenates Excel files (.xlsx), saving the result into a single Excel file.

    :param file_paths: List of input Excel file paths.
    :param output_path: Path to save the concatenated Excel file.
    """
    dataframes = []

    for path in file_paths:
        try:
            df = pd.read_excel(path)
            dataframes.append(df)
            print(f"Excel carrgado: {path}")

        except Exception as e:
            print(f"Error reading {path}: {e}")

    if dataframes:
        df_final = pd.concat(dataframes, ignore_index=True)
        df_final.to_excel(output_path, index=False)
        print(f"Arquivo final salvo em: {output_path}")

    else:
        print("Nenhum arquivo válido foi carregado.")

## Utilitário

In [4]:
def list_file_paths(directory: str) -> list[str]:
    """
    Retorna uma lista com os caminhos completos de todos os arquivos em um diretório.

    :param directory: Caminho do diretório a ser listado.
    :return: Lista de caminhos completos dos arquivos.
    """
    return [
        os.path.join(directory, file)
        for file in os.listdir(directory)
        if os.path.isfile(os.path.join(directory, file))
    ]

# SETUP DO AMBIENTE

In [5]:
drive.mount("/content/drive")

Mounted at /content/drive


# SETUP

In [6]:
# Constantes simbólicas
PRE = "pre_processing"              # Pré-processamento
RF = "random_forest"                # Random Forest
XGB = "xgboost"                     # XGBoost
MLR = "multiple_linear_regression"  # Regressão Linear Múltipla
ALL = "all"                         # Todos os modelos considerados simultaneamente

# CONCATENAÇÃO

## Subcenários Paralelizados

In [7]:
# SALVAMENTO
PATH_IN = "/content/drive/MyDrive/07_per_shared/projCDat_25_1/src/03_experiment_plan/plans"
PATH_OUT = "/content/drive/MyDrive/07_per_shared/projCDat_25_1/src/03_experiment_plan/plans"

CONCATENATING = [RF, XGB]
SUBSCENARIO_PREFIX = "{subscenario}_"

# Concatena os subcenários:
for model in CONCATENATING:
    print(f"\n\n### MODELO: {model} ###\n")

    # Obtém caminho de entrada
    path_in = os.path.join(PATH_IN, model)

    # Obtém todos os arquivos no caminho
    all_files = list_file_paths(path_in)

    # Obtém subcenários
    subscenarios_csv = list()
    subscenarios_excel = list()
    for file in all_files:
        file_name = os.path.basename(file)

        if file_name.startswith(SUBSCENARIO_PREFIX.format(subscenario=model)):

            if file_name.endswith(".csv"):
                subscenarios_csv.append(file)

            elif file_name.endswith(".xlsx"):
                subscenarios_excel.append(file)

    # Concatena os .csvs e salva em um único .csv consolidado
    path_out_csv = os.path.join(PATH_OUT, model, f"{model}.csv")
    concatenate_csvs(subscenarios_csv, path_out_csv)
    print("")

    # Concatena os .xlsx e salva em um único .xlsx consolidado
    path_out_excel = os.path.join(PATH_OUT, model, f"{model}.xlsx")
    concatenate_excels(subscenarios_excel, path_out_excel)



### MODELO: random_forest ###

CSV carregado: /content/drive/MyDrive/07_per_shared/projCDat_25_1/src/03_experiment_plan/plans/random_forest/random_forest_100_6.csv
CSV carregado: /content/drive/MyDrive/07_per_shared/projCDat_25_1/src/03_experiment_plan/plans/random_forest/random_forest_200_6.csv
CSV carregado: /content/drive/MyDrive/07_per_shared/projCDat_25_1/src/03_experiment_plan/plans/random_forest/random_forest_100_10.csv
CSV carregado: /content/drive/MyDrive/07_per_shared/projCDat_25_1/src/03_experiment_plan/plans/random_forest/random_forest_200_10.csv
Arquivo final salvo em: /content/drive/MyDrive/07_per_shared/projCDat_25_1/src/03_experiment_plan/plans/random_forest/random_forest.csv

Excel carrgado: /content/drive/MyDrive/07_per_shared/projCDat_25_1/src/03_experiment_plan/plans/random_forest/random_forest_100_6.xlsx
Excel carrgado: /content/drive/MyDrive/07_per_shared/projCDat_25_1/src/03_experiment_plan/plans/random_forest/random_forest_100_10.xlsx
Excel carrgado: /content/

## Plano de Experimentação

In [8]:
EXP_PLAN_NAME = f"experiment_plan_{ALL}"
EXP_PLAN_PATH_IN = "/content/drive/MyDrive/07_per_shared/projCDat_25_1/src/03_experiment_plan/plans"
EXP_PLAN_PATH_OUT = "/content/drive/MyDrive/07_per_shared/projCDat_25_1/src/03_experiment_plan/plans"
EXP_PLAN_PATH_OUT_CSV = os.path.join(EXP_PLAN_PATH_OUT, f"{EXP_PLAN_NAME}.csv")
EXP_PLAN_PATH_OUT_EXCEL = os.path.join(EXP_PLAN_PATH_OUT, f"{EXP_PLAN_NAME}.xlsx")

# Tipos de modelo a comporem o Plano de Experimentação
MODELS = [RF, XGB]

# Listas de arquivos
paths_csv = list()
paths_excel = list()

# Define planilhas a comporem o plano de experimentação
for model in MODELS:
    base_path = os.path.join(EXP_PLAN_PATH_IN, model)

    path_csv = os.path.join(base_path, f"{model}.csv")
    path_excel = os.path.join(base_path, f"{model}.xlsx")

    paths_csv.append(path_csv)
    paths_excel.append(path_excel)

# Salva em formato csv
print("### Plano de experimentação em CSV ###")
concatenate_csvs(paths_csv, EXP_PLAN_PATH_OUT_CSV)
print("")

# Salva em formato excel
print("### Plano de experimentação em Excel ###")
concatenate_excels(paths_excel, EXP_PLAN_PATH_OUT_EXCEL)

### Plano de experimentação em CSV ###
CSV carregado: /content/drive/MyDrive/07_per_shared/projCDat_25_1/src/03_experiment_plan/plans/random_forest/random_forest.csv
CSV carregado: /content/drive/MyDrive/07_per_shared/projCDat_25_1/src/03_experiment_plan/plans/xgboost/xgboost.csv
Arquivo final salvo em: /content/drive/MyDrive/07_per_shared/projCDat_25_1/src/03_experiment_plan/plans/experiment_plan_all.csv

### Plano de experimentação em Excel ###
Excel carrgado: /content/drive/MyDrive/07_per_shared/projCDat_25_1/src/03_experiment_plan/plans/random_forest/random_forest.xlsx
Excel carrgado: /content/drive/MyDrive/07_per_shared/projCDat_25_1/src/03_experiment_plan/plans/xgboost/xgboost.xlsx
Arquivo final salvo em: /content/drive/MyDrive/07_per_shared/projCDat_25_1/src/03_experiment_plan/plans/experiment_plan_all.xlsx
